In [1]:
# [1]*10**10

Handle Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import dataset files from Google Drive

In [4]:
z = zipfile.ZipFile("/content/drive/MyDrive/YouTube-Spam-Collection-v1.zip")
Psy = pd.read_csv(z.open("Youtube01-Psy.csv"))
Katy = pd.read_csv(z.open("Youtube02-KatyPerry.csv"))
LMFAO = pd.read_csv(z.open("Youtube03-LMFAO.csv"))
Eminem = pd.read_csv(z.open("Youtube04-Eminem.csv"))
Shakira = pd.read_csv(z.open("Youtube05-Shakira.csv"))

In [5]:
data = pd.concat([Psy, Katy, LMFAO, Eminem, Shakira])
data.drop(["COMMENT_ID", "DATE", "AUTHOR"], axis=1, inplace=True)

data.shape
data.head(5)

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 369
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CONTENT  1956 non-null   object
 1   CLASS    1956 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.8+ KB


Splitting dataset into train/test sets

In [109]:
X_train, X_test, y_train, y_test = train_test_split(data["CONTENT"], data["CLASS"]) # uses 75% train and 25% test split by default

In [110]:
# tokenizing text and removing stop-words
vectorizer = CountVectorizer(lowercase=True)
X_train_counts = vectorizer.fit_transform(X_train)
# print(X_train_counts[1,:])

In [111]:
print(vectorizer.get_stop_words())

None


In [112]:
tf_transformer = TfidfTransformer(use_idf=True)
X_train_tfidf = tf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1467, 3711)

In [113]:
# training the Multinomial Naive Bayes model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [117]:
# perform same feature extraction on test set
X_test_counts = vectorizer.transform(X_test)
X_test_tfidf = tf_transformer.transform(X_test_counts)

# make predictions on the test set
predictions = model.predict(X_test_tfidf)

In [118]:
# generate confusion matrix
confusion_matrix(y_test, predictions)

array([[207,  23],
       [ 12, 247]])

In [119]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92       230
           1       0.91      0.95      0.93       259

    accuracy                           0.93       489
   macro avg       0.93      0.93      0.93       489
weighted avg       0.93      0.93      0.93       489



In [120]:
# exporting model to pickle file
with open("model.pkl", "wb") as model_file:
  pickle.dump(model, model_file)

In [121]:
# loading the model
with open("model.pkl", "rb") as model_file:
  loaded_model = pickle.load(model_file)
loaded_model
loaded_model.score(X_test_tfidf, y_test)

0.9284253578732107

In [122]:
# testing the model with our own test comments
comment = 'facebook.com'
test_df = pd.DataFrame(data=[[comment]], columns=['CONTENT'])
test_df

test_comm_counts = vectorizer.transform(test_df['CONTENT']) # TODO: FIGURE OUT why we need access 'content' here..
# vectorizer.get_feature_names()
# test_comm_counts.toarray() # log and learn what this actually represents!
test_comm_tfidf = tf_transformer.transform(test_comm_counts) # learn the diff between this and test_comm_counts
loaded_model.predict(test_comm_tfidf)

# TODO: need to export vectorizer and reload?
# https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python


array([1])

In [123]:
data.loc[data['CLASS'] == 0]
# data.iloc[7]
correct_preds_inds = np.where(loaded_model.predict(X_test_tfidf) == y_test)
# correct_preds_inds[0]
correct_preds_ham_inds = np.where(y_test.iloc[correct_preds_inds[0]] == 0)
correct_preds_ham_inds
correct_preds = X_test.iloc[correct_preds_ham_inds[0]]
correct_preds
# X_test.iloc[0]
# data.iloc[53]

98                               Are those real animals﻿
78     This song is special, because is a song for Af...
155    What free gift cards? Go here  http://www.swag...
198                              Check out my channel :)
158                              I like this song<br />﻿
                             ...                        
116                                           waka waka﻿
6      Subscribe pleaaaase to my instagram account , ...
96     this video is very inaccurate, a tiger would r...
37                                       Love this song﻿
24     I really love this video.. http://www.bubblews...
Name: CONTENT, Length: 207, dtype: object

In [128]:
# (loaded_model.predict(X_test_tfidf) == y_test).index
# data.loc[237]

In [127]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(loaded_model, X_train_tfidf, y_train, cv=10)
cv_scores.mean()

# cv example:
# https://medium.com/@akshmahesh/detecting-spam-comments-on-youtube-using-machine-learning-948d54f47b3
# https://github.com/AkshayLaddha943/Machine-Learning/blob/master/Youtube-Spam-Check/youtube-spam.py

0.9215869909607679